In [4]:
#imports
import re
import string
import numpy as np 
import pandas as pd

In [5]:
from bs4 import BeautifulSoup
import requests
from lxml import html 
page = requests.get('https://www.seed-db.com/accelerators/viewall?acceleratorid=1011')

soup = BeautifulSoup(page.text, 'html.parser')

companies = soup.find('table', {'id': 'seedcos'})
tbody = companies.find('tbody')

# lists to append to for dataframe creation
list0 = []
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []

# finding the elements in tbody 
for tr in tbody.find_all('tr'):
    status = tr.find_all('td')[0].text.strip()
    list0.append(status)
    names = tr.find_all('td')[1].text.strip()
    list1.append(names)
    websites = tr.find_all('td')[2].text.strip()
    list2.append(websites)
    cohort_dates = tr.find_all('td')[3].text.strip()
    list3.append(cohort_dates)
    exit_values = tr.find_all('td')[4].text.strip()
    list4.append(exit_values)
    total_funding = tr.find_all('td')[6].text.strip()
    list5.append(total_funding)

In [6]:
df = pd.DataFrame(data=[list3, list0, list1, list2, list4, list5])
df.T

yc = df.T
print(yc.shape)
yc.head()

(1801, 6)


,0,1,2,3,4,5
0,6/2010,Exited,Rapportive,http://rapportive.com,"$ 15,000,000","$ 1,000,000"
1,1/2006,Exited,Snipshot,http://snipshot.com,,
2,1/2006,Exited,Clustrix,http://www.clustrix.com,,"$ 71,650,000"
3,6/2006,Exited,Xobni,http://www.xobni.com,"$ 48,000,000","$ 41,752,000"
4,1/2008,Dead,Kirkland North,http://www.playturf.net,,"$ 225,000"


In [7]:
# rename columns 
yc = yc.rename(columns={0:'cohort_date', 1:'status', 2:'company', 3:'website', 4:'exit_value', 5:'total_funding'})
yc.head()

,cohort_date,status,company,website,exit_value,total_funding
0,6/2010,Exited,Rapportive,http://rapportive.com,"$ 15,000,000","$ 1,000,000"
1,1/2006,Exited,Snipshot,http://snipshot.com,,
2,1/2006,Exited,Clustrix,http://www.clustrix.com,,"$ 71,650,000"
3,6/2006,Exited,Xobni,http://www.xobni.com,"$ 48,000,000","$ 41,752,000"
4,1/2008,Dead,Kirkland North,http://www.playturf.net,,"$ 225,000"


In [6]:
# use date to create yc cohort year with Winter and Summer cohorts 
yc['yc_cohort'] = yc['cohort_date'].replace(
    {'6/2005': 'YC S05',
     '1/2006': 'YC W06',
     '6/2006': 'YC S06',
     '1/2007': 'YC W07',
     '6/2007': 'YC S07',
     '1/2008': 'YC W08',
     '6/2008': 'YC S08',
     '1/2009': 'YC W09',
     '6/2009': 'YC S09',
     '1/2010': 'YC W10',
     '6/2010': 'YC S10',
     '1/2011': 'YC W11',
     '6/2011': 'YC S11',
     '1/2012': 'YC W12',
     '6/2012': 'YC S12',
     '1/2013': 'YC W13',
     '6/2013': 'YC S13',
     '1/2014': 'YC W14',
     '6/2014': 'YC S14',
     '1/2015': 'YC W15',
     '6/2015': 'YC S15',
     '1/2016': 'YC W16',
     '6/2016': 'YC S16',
     '1/2017': 'YC W17',
     '6/2017': 'YC S17',
     '1/2018': 'YC W18',
     '6/2018': 'YC S18',  
     })

In [8]:
yc['cohort_date'] = pd.to_datetime(yc['cohort_date'], infer_datetime_format=True)

In [8]:
yc['cohort_date'].value_counts()

2018-06-01    260
2018-01-01    141
2017-06-01    124
2016-01-01    123
2015-01-01    113
2015-06-01    107
2017-01-01    103
2016-06-01     96
2012-06-01     87
2014-06-01     86
2014-01-01     75
2012-01-01     65
2011-06-01     64
2013-06-01     55
2013-01-01     48
2011-01-01     45
2010-06-01     36
2009-06-01     29
2010-01-01     27
2008-06-01     23
2008-01-01     20
2007-06-01     19
2009-01-01     16
2007-01-01     13
2006-06-01     10
2005-06-01      9
2006-01-01      7
Name: cohort_date, dtype: int64

In [9]:
# sort the dataframe by date
yc = yc.sort_values(by="cohort_date")

In [10]:
yc.head()

,cohort_date,status,company,website,exit_value,total_funding
355,2005-06-01,Exited,Loopt,http://www.loopt.com,"$ 43,400,000","$ 39,105,997"
23,2005-06-01,Exited,Reddit,http://www.reddit.com,"$ 20,000,000","$ 550,100,000"
24,2005-06-01,Dead,Simmery,None,,
25,2005-06-01,Exited,Parakey,http://www.parakey.com,"$ 5,000,000",
270,2005-06-01,,Infogami,http://infogami.co.uk,,


In [11]:
yc['total_funding'] = yc['total_funding'].str.strip("$ ")
yc['total_funding'] = yc['total_funding'].str.replace(',','')
yc['total_funding'] = yc['total_funding'].replace(r'', np.nan)
yc['total_funding'] = yc['total_funding'].replace(np.nan, 0)
yc['total_funding'] = yc['total_funding'].astype(float)
yc['total_funding']

355      39105997.0
23      550100000.0
24              0.0
25              0.0
270             0.0
           ...     
1335       120000.0
720      13205000.0
1014       120000.0
977       1620000.0
1057      1220000.0
Name: total_funding, Length: 1801, dtype: float64

In [12]:
yc['exit_value'] = yc['exit_value'].str.strip("$ ")
yc['exit_value'] = yc['exit_value'].str.replace(',','')
yc['exit_value'] = yc['exit_value'].replace(r'', np.nan)
yc['exit_value'] = yc['exit_value'].replace(np.nan, 0)
yc['exit_value'] = yc['exit_value'].astype(float)
yc['exit_value']

355     43400000.0
23      20000000.0
24             0.0
25       5000000.0
270            0.0
           ...    
1335           0.0
720            0.0
1014           0.0
977            0.0
1057           0.0
Name: exit_value, Length: 1801, dtype: float64

In [13]:
print('YC companies total funding:', yc['total_funding'].sum())
print('YC companies total exit value:', yc['exit_value'].sum())

YC companies total funding: 40121135290.0
YC companies total exit value: 6195483100.0


In [14]:
company_exited = yc['status']=='Exited'
company_failed = yc['status']=='Dead'

print('YC total companies:', yc.shape[0])
print('YC failed companies:',company_failed.sum())
print('YC exited companies:',company_exited.sum())

YC total companies: 1801
YC failed companies: 143
YC exited companies: 254


In [15]:
yc.to_csv('yc.csv',index=False)

In [16]:
import plotly.express as px

In [183]:
# total funding by yc_cohort, still tbd on younger cohorts
px.bar(yc, 'cohort_date', 'total_funding', color='yc_cohort')

In [17]:
px.scatter(yc, 'total_funding', 'exit_value', color='company')

In [18]:
top100 = yc[['total_funding', 'company', 'exit_value', 'yc_cohort']].sort_values(by='total_funding', ascending=False)[:100]

KeyError: "['yc_cohort'] not in index"

In [ ]:
top100['yc_cohort'].value_counts()

In [18]:
fig = px.bar(top100, 'total_funding', 'company', color='yc_cohort', orientation='h')
fig.update_yaxes(automargin=True)
fig.show()

In [ ]:
gapminder = px.data.gapminder()
fig = px.scatter(yc, x="total_funding", y="yc_cohort", size="total_funding", color="yc_cohort",
           hover_name="company")

fig.show()

# looking at top 100

In [ ]:
gapminder = px.data.gapminder()
fig = px.scatter(top100, x="total_funding", y="yc_cohort", size="total_funding", color="yc_cohort",
           hover_name="company")
fig.update_yaxes(automargin=True)

fig.show()

In [21]:
yc.head()

,cohort_date,status,company,website,exit_value,total_funding,yc_cohort
355,2005-06-01,Exited,Loopt,http://www.loopt.com,43400000.0,39105997.0,YC S05
23,2005-06-01,Exited,Reddit,http://www.reddit.com,20000000.0,550100000.0,YC S05
24,2005-06-01,Dead,Simmery,None,0.0,0.0,YC S05
25,2005-06-01,Exited,Parakey,http://www.parakey.com,5000000.0,0.0,YC S05
270,2005-06-01,,Infogami,http://infogami.co.uk,0.0,0.0,YC S05


In [ ]:
def plot_cohort(df):
    gapminder = px.data.gapminder()
    fig = px.scatter(df, x="total_funding", y="company", size="total_funding", color="yc_cohort",
           hover_name="company")
    fig.update_yaxes(automargin=True)
    return fig.show()

In [25]:
# create a df for every cohort
S05 = yc[yc['yc_cohort'] == 'YC S05']
W06 = yc[yc['yc_cohort'] == 'YC W06']
S06 = yc[yc['yc_cohort'] == 'YC S06']
W07 = yc[yc['yc_cohort'] == 'YC W07']
S07 = yc[yc['yc_cohort'] == 'YC S07']
W08 = yc[yc['yc_cohort'] == 'YC W08']
S08 = yc[yc['yc_cohort'] == 'YC S08']
W09 = yc[yc['yc_cohort'] == 'YC W09']
S09 = yc[yc['yc_cohort'] == 'YC S09']
W10 = yc[yc['yc_cohort'] == 'YC W10']
S10 = yc[yc['yc_cohort'] == 'YC S10']
W11 = yc[yc['yc_cohort'] == 'YC W11']
S11 = yc[yc['yc_cohort'] == 'YC S11']
W12 = yc[yc['yc_cohort'] == 'YC W12']
S12 = yc[yc['yc_cohort'] == 'YC S12']
W13 = yc[yc['yc_cohort'] == 'YC W13']
S13 = yc[yc['yc_cohort'] == 'YC S13']
W14 = yc[yc['yc_cohort'] == 'YC W14']
S14 = yc[yc['yc_cohort'] == 'YC S14']
W15 = yc[yc['yc_cohort'] == 'YC W15']
S15 = yc[yc['yc_cohort'] == 'YC S15']
W16 = yc[yc['yc_cohort'] == 'YC W16']
S16 = yc[yc['yc_cohort'] == 'YC S16']
W17 = yc[yc['yc_cohort'] == 'YC W17']
S17 = yc[yc['yc_cohort'] == 'YC S17']
W18 = yc[yc['yc_cohort'] == 'YC W18']
S18 = yc[yc['yc_cohort'] == 'YC S18']

In [ ]:
def bar_graph_cohort(df):
    gapminder = px.data.gapminder()
    fig = px.bar(df, x="total_funding", y="company", color="yc_cohort",
           hover_name="company", orientation = 'h')
    fig.update_yaxes(automargin=True)
    return fig.show()

In [232]:
bar_graph_cohort(S09)

In [ ]:
plot_cohort(W16)